In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import pandas as pd
import numpy as np
from os import path
import matplotlib.pyplot as plt

In [ ]:
data_file = ""

In [ ]:
df = pd.read_csv(path.join("..", "cleaned_data", data_file))
y = df["label"]
X = df.drop(columns=["label"])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

In [ ]:
models = [("XGBoost", XGBClassifier(enable_categorical=True)), ]

In [ ]:
min_error = float("inf")

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmsle_score = rmsle(y_test, y_pred)

    if rmsle_score < min_error:
        min_error = rmsle_score
        best_model = (name, model)

    print(f"{name}: RMSLE = {rmsle_score}")
print(best_model[0])